In [11]:
from ase.io import read, write
from mace.calculators import mace_mp
from ase.build import surface
from ase.visualize import view
from ase.constraints import FixAtoms, UnitCellFilter
from ase.build import bulk
import numpy as np
from ase.optimize import LBFGS

x = read('/home/lana/Downloads/9007497.cif')
x.calc = mace_mp(model="medium", dispersion=False, default_dtype="float64", device='cpu')
ucf = UnitCellFilter(x)
qn = LBFGS(ucf, trajectory='bulk_rlx.traj')
qn.run(fmax=0.01)
E_x = x.get_potential_energy()
t = read('bulk_rlx.traj@:')
view(t)

Using Materials Project MACE for MACECalculator with /home/lana/.cache/mace/5yyxdm76
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 15:07:04      -28.749845*       0.0015


<Popen: returncode: None args: ['/home/lana/ve/bin/python3', '-m', 'ase', 'g...>

In [4]:
from ase import Atoms
from mace.calculators import mace_mp
from ase.build import surface
from ase.constraints import FixAtoms, UnitCellFilter
from ase.optimize import QuasiNewton
from ase.visualize import view
from ase.io import read, write
from ase.optimize import LBFGS
import numpy as np
from cu2o_bulk import cu2o_bulk, cu2o111, STO_FCC111

b = cu2o_bulk()

b.calc = mace_mp(model="light", dispersion=False, default_dtype="float64", device='cpu')
ucf = UnitCellFilter(b)
LBFGS(ucf).run(fmax=0.01)


#E_bulk = b.get_potential_energy()


#slab = cu2o111(5, 10)
#slab.calc = mace_mp(model="medium", dispersion=False, default_dtype="float64", device='cpu')
#view(slab)


slab = STO_FCC111(b, 3, 10)
view(slab)


slab.calc = mace_mp(model="light", dispersion=False, default_dtype="float64", device='cpu')

n_layers=3
E_slab = slab.get_potential_energy()
E_cleav = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])
print(f'{n_layers=} {E_cleav=}')

bottom_Cu_z = np.min(slab[slab.symbols=='Cu'].positions[:,2])
mask1=slab.positions[:, 2] < bottom_Cu_z + 1.0
slab.set_constraint(FixAtoms(mask=mask1))

qn = LBFGS(slab, trajectory='111slab.traj')
qn.run(fmax=0.01)
E_slab = slab.get_potential_energy()
t = read('111slab.traj@:')
atoms = t[-1]
view(atoms)

E_surf = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])
print(f'{n_layers=} {E_surf=}')

16


"\nE_slab = slab.get_potential_energy()\nE_cleav = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])\nprint(f'{n_layers=} {E_cleav=}')\n\nbottom_Cu_z = np.min(slab[slab.symbols=='Cu'].positions[:,2])\nmask1=slab.positions[:, 2] < bottom_Cu_z + 1.0\nslab.set_constraint(FixAtoms(mask=mask1))\n\nqn = LBFGS(slab, trajectory='111slab.traj')\nqn.run(fmax=0.01)\nE_slab = slab.get_potential_energy()\nt = read('111slab.traj@:')\natoms = t[-1]\nview(atoms)\n\nE_surf = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])\nprint(f'{n_layers=} {E_surf=}')\n"